In [ ]:
%load_ext autoreload
%autoreload 2
%env LD_LIBRARY_PATH=$HOME/cfcx/recourse/model/LORE/yadt:$LD_LIBRARY_PATH
    
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import make_dot
import pickle
from lux.lux import LUX
from sklearn.preprocessing import StandardScaler

In [ ]:
def to_latex(aggregated, aggregated_std, index_name='', caption='',label=''):
    aggregated=aggregated.replace('_','\_')
    aggregated_std=aggregated_std.replace('_','\_')
    print('\\begin{table}')
    print('\\caption{'+caption+'}')
    print('\\label{'+label+'}')
    print('\\begin{tabularx}{\\textwidth}{|X|'+'|'.join(['X']*len(aggregated.columns))+'|}')
    print('\\hline')
    print(index_name+' & '+'&'.join(aggregated.columns))
    print('\\\\ \\hline \\hline')
    for i,row in aggregated.iterrows():
        row_std = aggregated_std.loc[i]
        rowstring = []
        for r,r_std in zip(row,row_std):
            rowstring.append("{:0.2f}".format(r)+' $\pm$ '+"{:0.2f}".format(r_std))
        print(str(i)+' & '+' & '.join(rowstring)+'\\\\ \hline')
    print('\\end{tabularx}\n\\end{table}')

In [ ]:
def to_latex_nostd(aggregated, index_name='', caption='',label=''):
    aggregated=aggregated.replace('_','\_')
    print('\\begin{table}')
    print('\\caption{'+caption+'}')
    print('\\label{'+label+'}')
    print('\\begin{tabularx}{\\textwidth}{|X|'+'|'.join(['X']*len(aggregated.columns))+'|}')
    print('\\hline')
    print(index_name+' & '+'&'.join(aggregated.columns))
    print('\\\\ \\hline \\hline')
    for i,row in aggregated.iterrows():
        rowstring = []
        for r in row:
            rowstring.append("{:0.2f}".format(r))
        print(str(i)+' & '+' & '.join(rowstring)+'\\\\ \hline')
    print('\\end{tabularx}\n\\end{table}')

In [ ]:
from sklearn.utils.extmath import softmax
class EBMCounterOptimizer:
    Y_TEST = '_eco_y_test'
    Y_PRED = '_eco_y_pred'
    IS_MODIFIABLE = '_eco_is_modifiable'

    def __init__(self, ebm, X):
        self.ebm = ebm
        self.X = X
        self.updated_features = {}
        
    def _get_optimized_feature_value(self, feature_name, feature_idx, feature_val, features, feature_masked, term_idx, class_idx):
        """
        Returns feature value with maximum score for given target class. 
        
        @Todo Needs changes to return optimized value due to given strategy. 
        """

        if feature_name in feature_masked and feature_name not in self.updated_features:
            if len(self.ebm.term_scores_[term_idx].shape) > 1:
                class_term_scores = self.ebm.term_scores_[term_idx].T[class_idx]
            else:
                class_term_scores =  self.ebm.term_scores_[term_idx] if class_idx == 1 else 1-self.ebm.term_scores_[term_idx]
            class_max = np.max(class_term_scores)
            feature_score_idx = np.where(class_term_scores[1:-1]==class_max)[0][0] ##this is score, not value imho
            # we bin differently for main effects and pairs, so first 
            # get the list containing the bins for different resolutions
            bin_levels = self.ebm.bins_[feature_idx]
            #print(f'Feature score index for feature {feature_name} is {feature_score_idx} which represents score equal: {class_max} test: {class_term_scores[feature_score_idx+1]}')
            # what resolution do we need for this term (main resolution, pair
            # resolution, etc.), but limit to the last resolution available
            bins = bin_levels[min(len(bin_levels), len(features)) - 1]
            
            if len(bins)==0:
                feature_val = self.X[feature_name].sample(1).values[0]
            else:
                if isinstance(bins, dict):
                    # categorical feature
                    # 'unknown' category strings are in the last bin (-1)
                    feature_val=list(bins.values())[feature_score_idx-1] # if maxscore was 0, or -1 just assign random value
                else:
                    # continuous feature
                    # Get the lower and upper bounds of the specified bin
                    lower_idx = feature_score_idx-1
                    upper_idx = feature_score_idx
                    
                    if lower_idx == -1:
                        lower = self.ebm.feature_bounds_[feature_idx][0]
                    else:
                        lower = bins[lower_idx]

                    if upper_idx == len(bins):
                        upper = self.ebm.feature_bounds_[feature_idx][1]
                    else:
                        upper = bins[upper_idx]
                    #print(f'Drawing randomly from :{lower} to {upper}')

                    # Draw a random number from the range defined by the bin
                    feature_val = np.random.uniform(lower, upper)

            #print(f'This translates into feature value: {feature_val}')
            
            self.updated_features.update({feature_name: feature_val})
        elif feature_name in self.updated_features:
            feature_val = self.updated_features.get(feature_name)
        else:
            self.updated_features.update({feature_name: feature_val})

        return feature_val
        

    def optimize_proba(self, target_class, feature_masked):
        """
        The method calculates probabilities taking into account the optimization of given parameters towards the target class.
        Method is based on a default ebm's predict_proba

        Parameters:
        ebm: Trained EBM model
        X: Dataset
        target_class: Target class from which we take the features
        featured_masked: List of interchangeable features
        """
        if target_class not in self.ebm.classes_:
            raise KeyError(f'Class "{target_class}" does not exists in given EBM model')

        class_idx = np.where(self.ebm.classes_==target_class)[0][0]
        self.updated_features = {}
        sample_scores = []
        cf = {}
        for index, sample in self.X.iterrows():
            # start from the intercept for each sample
            score = self.ebm.intercept_.copy()
            if isinstance(score, float) or len(score) == 1:
                # regression or binary classification
                score = float(score)

            # we have 2 terms, so add their score contributions
            for term_idx, features in enumerate(self.ebm.term_features_):
                # indexing into a tensor requires a multi-dimensional index
                tensor_index = []
                # main effects will have 1 feature, and pairs will have 2 features
                for feature_idx in features:
                    feature_name = self.ebm.feature_names_in_[feature_idx]  # Get the feature name by index
                    feature_val = sample[feature_name]  # Use the feature name to get the correct value from the sample
                    bin_idx = 0  # if missing value, use bin index 0

                    if feature_val is not None and feature_val is not np.nan:
                        # we bin differently for main effects and pairs, so first 
                        # get the list containing the bins for different resolutions
                        bin_levels = self.ebm.bins_[feature_idx]

                        # what resolution do we need for this term (main resolution, pair
                        # resolution, etc.), but limit to the last resolution available
                        bins = bin_levels[min(len(bin_levels), len(features)) - 1]

                        
                        # here is where the magic is located
                        feature_val = self._get_optimized_feature_value(feature_name, feature_idx, feature_val, features, feature_masked, term_idx, class_idx)

                        if isinstance(bins, dict):
                            # categorical feature
                            # 'unknown' category strings are in the last bin (-1)
                            bin_idx = bins.get(feature_val, -1)
                            if bin_idx == -1:
                                # check value as string
                                bin_idx = bins.get(str(feature_val), -1)
                        else:
                            # continuous feature
                            try:
                                # try converting to a float, if that fails it's 'unknown'
                                feature_val = float(feature_val)
                                # add 1 because the 0th bin is reserved for 'missing'
                                bin_idx = np.digitize(feature_val, bins) + 1
                            except ValueError:
                                # non-floats are 'unknown', which is in the last bin (-1)
                                bin_idx = -1
                                
                        if len(self.ebm.term_scores_[term_idx].shape) > 1:
                            sc = self.ebm.term_scores_[term_idx].T[class_idx][bin_idx]
                        else:
                            sc = self.ebm.term_scores_[term_idx][bin_idx]
                        #print(f'And feature value {feature_val} translates back to bin index: {bin_idx} which represents score: {sc}')
                        
                        tensor_index.append(bin_idx)
                

                # local_score is also the local feature importance
                local_score = self.ebm.term_scores_[term_idx][tuple(tensor_index)]

                score += local_score
            sample_scores.append(score)

        predictions = np.array(sample_scores)

        if hasattr(self.ebm, 'classes_'):
            # classification
            if len(self.ebm.classes_) == 2:
                # binary classification

                # softmax expects two logits for binary classification
                # the first logit is always equivalent to 0 for binary classification
                predictions = [[0, x] for x in predictions]
            predictions = softmax(predictions)

        return predictions,self.updated_features
    
    def check_samples(self, target_class, y_test_key, feature_masked):
        X = self.X.copy()
        predictions = self.optimize_proba(target_class, feature_masked)
        X.loc[:, self.Y_TEST] = np.argmax(predictions, axis=1)
        X.loc[:, self.Y_PRED] = X[self.Y_TEST].map({key:val for key, val in enumerate(self.ebm.classes_)})
        X.loc[:, self.IS_MODIFIABLE] = np.where(X[y_test_key]!=X[self.Y_PRED], 1, 0)
        
        return X

## Causal counterfactuals

In [ ]:
def generate_sample_from_csf(adjacency_matrix, csf, sample_order=None, categorical=None):
    num_vars = adjacency_matrix.shape[0]
    samples = np.zeros((1, num_vars))
    
    if categorical is None:
        categorical = [False] * num_vars  # Default to all variables being non-categorical

    if sample_order is None:
        sample_order = range(num_vars)

    # Assign noise values to variables without parents

    for j in range(num_vars):
        if np.all(adjacency_matrix[j, :] == 0):  # If no parents, assign noise
            samples[0, j] = csf[j]
            if categorical[j]:  # Round the value if the variable is categorical
                samples[0, j] = np.round(samples[0, j])


    # Generate samples based on the specified order
    for j in sample_order:
        parents = np.where(adjacency_matrix[j, :] != 0)[0]
        if len(parents) > 0:
            # Predicted values based on parents
            predicted_values = np.dot(samples[:, parents], adjacency_matrix[j, parents])
            samples[:, j]=predicted_values
            if categorical[j]:  # Round the value if the variable is categorical
                samples[:, j] = np.round(samples[:, j])
            
    return samples

In [ ]:
def get_parents(adjacency_matrix):
    parents = []
    num_vars = adjacency_matrix.shape[0]
    for j in range(num_vars):
        if np.all(adjacency_matrix[j, :] == 0):
            parents.append(j)
    return parents

In [ ]:
def generate_samples(adjacency_matrix, num_samples, bounds = None,  sample_order=None, categorical=None):
    num_vars = adjacency_matrix.shape[0]
    samples = np.zeros((num_samples, num_vars))
    
    if categorical is None:
        categorical = [False] * num_vars  # Default to all variables being non-categorical

    if sample_order is None:
        sample_order = range(num_vars)

    # Assign noise values to variables without parents
    for i in range(num_samples):
        if bounds is None:
            noise = np.random.normal(size=num_vars) #assign_csf
        else:
            noise = np.random.normal(size=num_vars)
        for j in range(num_vars):
            if np.all(adjacency_matrix[j, :] == 0):  # If no parents, assign noise
                samples[i, j] = noise[j]
                if categorical[j]:  # Round the value if the variable is categorical
                    samples[i, j] = np.round(samples[i, j])


    # Generate samples based on the specified order
    for j in sample_order:
        parents = np.where(adjacency_matrix[j, :] != 0)[0]
        if len(parents) > 0:
            # Predicted values based on parents
            predicted_values = np.dot(samples[:, parents], adjacency_matrix[j, parents])
            samples[:, j]=predicted_values
            if categorical[j]:  # Round the value if the variable is categorical
                samples[:, j] = np.round(samples[:, j])
            
    return samples

In [ ]:
#TODO: in such calcualtion , the difference in cat feature should always be equal to 1 as a penalty
def compute_causal_penalty(samples, adjacency_matrix, sample_order, categorical=None):
    """
    Calculate the inconsistency of samples with the given adjacency matrix.
    
    Parameters:
    - adjacency_matrix: np.ndarray, the adjacency matrix from DirectLiNGAM
    - samples: np.ndarray, the samples to evaluate (num_samples x num_features)
    
    Returns:
    - inconsistency: float, a measure of how inconsistent the samples are with the adjacency matrix
    """
    num_samples, num_features = samples.shape
    inconsistency = 0.0

    if categorical is None:
        categorical = [False]*len(sample_order)
    # Iterate through each feature and its causal parents
    for i in sample_order:
        parents = np.where(adjacency_matrix[i, :] != 0)[0]
        if len(parents) > 0:
            # Predicted values based on parents
            predicted_values = np.dot(samples[:, parents], adjacency_matrix[i, parents])
            # Calculate the inconsistency as the mean squared error
            mse = (samples[:, i] - predicted_values) ** 2
            if categorical[i]:
                mse = min((1,mse))
            inconsistency += mse

    return np.sqrt(np.mean(inconsistency))/len(sample_order)

In [ ]:
def compute_yloss(model, cfs, desired_class):
    predicted_value = np.array(model.predict_proba(cfs))
    maxvalue = np.full((len(predicted_value)), -np.inf)
    yloss=0
    for c in range(len(model.classes_)):
        #print(c)
        if c != desired_class:
            maxvalue = np.maximum(maxvalue, predicted_value[:, c])
    #print(predicted_value)
    #print(maxvalue)
    yloss = np.maximum(0, maxvalue - predicted_value[:, int(desired_class)])
    return yloss,maxvalue

In [ ]:
#data has to be normalized and proximity will be consine similarity in that casse

def compute_proximity_loss(cfs, query_instance, pbounds, features_order = None, feature_weights=None, categorical=None):
    """Compute weighted distance between two vectors."""
    # If feature_weights is None, assign an array of ones with the size of cfs.shape[1]
    
    if feature_weights is None:
        feature_weights = np.ones(cfs.shape[1])
    if categorical is None:
        categorical = [False]*cfs.shape[1]
    
    diff = abs(cfs - query_instance)
    diff=[1 if categorical[i] and v > 0 else v for i, v in enumerate(diff[0, :])]
    
    diff=[v/(pbounds[features_order[i]][1]-pbounds[features_order[i]][0]) if not categorical[i] and features_order[i] in pbounds.keys() else v for i, v in enumerate(diff)]
    product = np.multiply(
        (diff),
        feature_weights)
    product = product.reshape(-1, product.shape[-1])
    proximity_loss = np.sum(product, axis=1)

    # Dividing by the sum of feature weights to normalize proximity loss
    return proximity_loss / sum(feature_weights)

In [ ]:
def compute_sparsity_loss(cfs,query_instance):
        """Compute weighted distance between two vectors."""
        sparsity_loss = np.count_nonzero(cfs - query_instance, axis=1)
        return sparsity_loss / cfs.shape[1]  # Dividing by the number of features to normalize sparsity loss

In [ ]:
from scipy.spatial.distance import pdist,squareform
def compute_diversity_loss(cfs,low=1e-6, high=1e-5):
    # Compute pairwise distances
    pairwise_distances = pdist(cfs)

    # Convert the distances to a square matrix form
    #TODO: gower distance?
    distance_matrix = squareform(pairwise_distances)
    
    perturbations = np.random.uniform(low=low, high=high, size=distance_matrix.shape[0])

    # Add the random perturbations to the diagonal elements of the transformed matrix
    np.fill_diagonal(distance_matrix, distance_matrix.diagonal() + perturbations)
    
    transformed_matrix = 1 / (1 + distance_matrix)
        
    # Calculate the determinant of the transformed matrix
    determinant = np.linalg.det(transformed_matrix) 
    return determinant

In [ ]:
def compute_loss(model, cfs, query_instance, desired_class, adjency_matrix, causal_order, 
                 proximity_weight, sparsity_weight,plausability_weight,diversity_weight, pbounds,features_order,
                 masked_features, categorical=None, allcfs=[]):
    """Computes the overall loss"""
    yloss,maxyloss = compute_yloss(model, cfs, desired_class)
    conditional_term = 1.0/(yloss+1+sum([proximity_weight, sparsity_weight,plausability_weight,diversity_weight])) 
    
    proximity_loss = compute_proximity_loss(cfs, query_instance,pbounds,features_order=features_order, categorical=categorical) \
        if proximity_weight > 0 else 0.0
    sparsity_loss = compute_sparsity_loss(cfs,query_instance) if sparsity_weight > 0 else 0.0
    plausability_loss = compute_causal_penalty(cfs,adjency_matrix,causal_order,categorical=categorical) if plausability_weight > 0 else 0
    diversity_loss =1-np.abs(compute_diversity_loss(allcfs)) if len(allcfs) > 1 and diversity_weight > 0 else 0
    #print(f'DL: {diversity_loss} for len of cfs {len(allcfs)}, and PrxL: {proximity_loss} and PL {plausability_loss} and sparl: {sparsity_loss}')
    
    loss = np.reshape(np.array(yloss + conditional_term*((diversity_loss*diversity_weight)+ (proximity_weight * proximity_loss) +
                                    (sparsity_weight * sparsity_loss) +(plausability_loss*plausability_weight))), (-1, 1))
    index = np.reshape(np.arange(len(cfs)), (-1, 1))
    loss = np.concatenate([index, loss], axis=1)
    return loss

In [ ]:
def print_cfs(qi, cfs, features):
    """
    Construct a DataFrame from a list of instances cfs,
    filling fields that are the same as in instance qi with '-'.
    
    Parameters:
    qi (dict): An instance represented as a dictionary.
    cfs (list of dicts): A list of instances, each represented as a dictionary.
    
    Returns:
    pd.DataFrame: Transformed DataFrame with matching fields replaced by '-'.
    """
    df = pd.DataFrame(cfs, columns=features)
    
    # Replace fields that are the same as in qi with '-'
    for col in df.columns:
        df[col] = df[col].apply(lambda x: '-' if x == qi[df.columns.get_loc(col)] else f'{qi[df.columns.get_loc(col)]}->{x}')
    
    
    return df

## Bayesian optimization for Causal counterfactual generation


In [ ]:
from bayes_opt import BayesianOptimization
import numpy as np
import random

In [ ]:
import optuna
import random
import numpy as np
optuna.logging.set_verbosity(optuna.logging.WARNING)

def generate_single_cf(query_instance, desired_class, adjacency_matrix, causal_order, proximity_weight, sparsity_weight, plausibility_weight, 
                       diversity_weight, bounds, model, 
                       categorical_indicator=None, features_order=None, masked_features=None, cfs=[], X=None, init_points=5, n_iter=1000):
    """
    Generate a single counterfactual that minimizes the loss function using Optuna.
    """
    if categorical_indicator is None:
        categorical_indicator = [False]*len(bounds)
    if features_order is None:
        features_order = list(bounds.keys())
    if masked_features is None:
        masked_features = features_order

    def update_masked_features_dict(features_order, masked_features, **params):
        for feature, value in zip(features_order, query_instance):
            if feature not in masked_features:
                params[feature] = value
        return params
        
    def black_box_function(trial):
        # Suggest values for each feature based on bounds
        kwargs = {}
        for i, key in enumerate(features_order):
            if categorical_indicator[i]:
                kwargs[key] = trial.suggest_categorical(key, list(range(int(bounds[key][0]), int(bounds[key][1])+1)))
            else:
                kwargs[key] = trial.suggest_uniform(key, bounds[key][0], bounds[key][1])

        kwargs = update_masked_features_dict(features_order, masked_features, **kwargs)
        #todo rounding
        cf = np.array([[int(round(kwargs[key])) if categorical_indicator[i] else kwargs[key] 
                        for i, key in enumerate(features_order)]]).reshape(1, -1)
        
        if len(cfs) > 0:
            cfst = np.vstack(cfs + [cf])
        else:
            cfst = cf

        loss = compute_loss(model, cf, query_instance, desired_class, adjacency_matrix, 
                            causal_order, proximity_weight, sparsity_weight, plausibility_weight,
                            diversity_weight, pbounds=bounds, features_order=features_order, masked_features=masked_features,
                            categorical=categorical_indicator, allcfs=cfst)
        # Return the first value in the loss array, Optuna needs a single scalar value to minimize
        return -loss[0, 1]

    # Initialize Optuna study
    study = optuna.create_study(direction='maximize')

    # Define seen_points set to track uniqueness of points
    seen_points = set()

    def is_unique_point(point_dict):
        point_tuple = tuple(point_dict[key] for key in features_order)
        if point_tuple in seen_points:
            return False
        seen_points.add(point_tuple)
        return True
    
    def clip_values(cf, pbounds):
        # Create a new dictionary with clipped values based on the bounds in pbounds
        return {feature: np.clip(value, pbounds[feature][0], pbounds[feature][1]) 
                for feature, value in cf.items()}



    sampled_trials = 0
    if X is not None:
        Xdesired = X[model.predict(X) == desired_class].drop_duplicates()
        sample_size = min(int(init_points * 0.5), len(Xdesired))
        Xsample = Xdesired.sample(sample_size)
        for i, r in Xsample.iterrows():
            candidate_point = dict(r[features_order])
            if is_unique_point(candidate_point):
                trial_params = {key: candidate_point[key] for key in features_order}
                study.enqueue_trial(trial_params)
                sampled_trials +=1
        init_points = max(0, init_points - sampled_trials)
        

    if init_points > 0:
        try:
            print(f'EBM random samples... Already sampled {sample_size} from {Xdesired.shape[0]} possible...')
            ebcf = EBMCounterOptimizer(model, pd.DataFrame(query_instance.reshape(1, -1), columns=features_order))
            total_lists = []
            for i in range(min(init_points, 2**len(masked_features))):
                num_elements = random.randint(1, len(masked_features))
                selected_features = random.sample(masked_features, num_elements)
                set_to_check = set(selected_features)
                # Check if the set is in the list of sets
                found = any(set(item) == set_to_check for item in total_lists)
                if found:
                    continue
                total_lists.append(selected_features)
                try:
                    _, cf = ebcf.optimize_proba(desired_class, feature_masked=selected_features)
                except:
                    continue

                #check if values of cf are aligned with the ranges, and if not, clip it to the range
                cf = clip_values(cf, bounds)

                if isinstance(cf, dict):
                    cf_dict = cf
                elif isinstance(cf, np.ndarray) or isinstance(cf, list):
                    cf_dict = {key: cf[i] for i, key in enumerate(features_order)}
                else:
                    raise ValueError("Unexpected format for cf returned by optimize_proba.")

                if is_unique_point(cf_dict):
                    sampled_trials += 1
                    study.enqueue_trial(cf_dict)
        except:
            print('Resampling failed...')

    # Optimize the study with the defined number of iterations
    study.optimize(black_box_function, n_trials=n_iter + sampled_trials)

    # Extract the best parameters (counterfactual)
    best_params = study.best_params
    best_params = update_masked_features_dict(features_order, masked_features, **best_params)
    #todo rounding should be done to the featoure boundaries, not to nearest integer
    best_cf = np.array([[int(round(best_params[key])) if categorical_indicator[i] else best_params[key] 
                        for i, key in enumerate(features_order)]]).reshape(1, -1)

    return best_cf

In [ ]:
def generate_cfs(query_instance, desired_class, adjacency_matrix, causal_order, proximity_weight, 
                 sparsity_weight, plausibility_weight, diversity_weight, bounds,  model, features_order, masked_features=None, categorical_indicator = None, X=None,
                 num_cfs=1, init_points=10, n_iter=1000):
    """
    Generate multiple counterfactuals that minimize the loss function using Bayesian Optimization.
    
    Parameters:
        query_instance: The instance to generate counterfactuals for.
        desired_class: The target class for the counterfactuals.
        adjacency_matrix: The adjacency matrix representing the causal structure.
        causal_order: The order of variables in the causal graph.
        proximity_weight, sparsity_weight, plausibility_weight: Weights for different loss components.
        bounds: The bounds for each feature to search over (dict with feature names as keys and tuple (min, max) as values).
        model: The predictive model used to predict class labels.
        categorical_indicator: True at the index where the variable should be treated as categorical
        num_cfs: The number of counterfactual instances to generate.
        init_points: Number of initial points for Bayesian Optimization.
        n_iter: Number of iterations for Bayesian Optimization.

    Returns:
        The generated counterfactuals that minimize the loss function.
    """
    cfs = []
    for _ in range(num_cfs):
        cf = generate_single_cf(query_instance, desired_class, adjacency_matrix, 
                                causal_order, proximity_weight, sparsity_weight, 
                                plausibility_weight, diversity_weight,
                                bounds, model,categorical_indicator,features_order, masked_features=masked_features,cfs=cfs,X=X,init_points=init_points, n_iter=n_iter)
        cfs.append(cf)

    return np.vstack(cfs)

In [ ]:
def get_feature_pbounds(ebm_model, feature_names, features_masked=None):
    bonds = dict([[f,(ebm_model.feature_bounds_[i][0],ebm_model.feature_bounds_[i][1])] for i,f in enumerate(feature_names)])
    if features_masked is None:
        return bonds
    else:
        return dict([[f,bonds[f]] for f in features_masked])

In [ ]:
def run_ccf(explain_instance, model_clf, dataset, desired_class, num_cfs, casual_model, pbounds, as_causal=True, masked_features=None, init_points=500, n_iter=100):
    return generate_cfs(explain_instance, desired_class=desired_class, adjacency_matrix=casual_model.adjacency_matrix_, causal_order=casual_model.causal_order_, 
                               proximity_weight=1, 
                               sparsity_weight=1, 
                               categorical_indicator = dataset._categorical_indicator,
                               plausibility_weight=int(as_causal), 
                               diversity_weight = 1,
                               bounds=pbounds, 
                               model=model_clf,
                               features_order = dataset.features,
                               masked_features = masked_features,
                               num_cfs=num_cfs,
                               X=dataset.df_train,
                               init_points=init_points, 
                               n_iter=n_iter)

## Evaluation on more datasets

In [ ]:
import openml
import numpy as np
import pandas as pd
import lingam
import time
from time import time as tstime
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import warnings
import random
import logging
import signal
import traceback

In [ ]:
from recourse.data import OpenmlData, CustomData, DataRecipe
from recourse.model import Dice

In [ ]:
def timeout_handler(signum, frame):
    raise TimeoutError("Calculation timed out!")

In [ ]:
# Register the timeout handler for the SIGALRM signal
signal.signal(signal.SIGALRM, timeout_handler)

In [ ]:
storage = './storage/'

In [ ]:
RANDOM_STATE=42
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [ ]:
logger = logging.getLogger('openml')
logger.handlers.clear()
logger.propagate = False
logging.getLogger('openml').setLevel(logging.ERROR)

In [ ]:
def log2file(output, filepath='progress.txt', clear=False):
    with open(filepath, 'w' if clear else 'a') as f:
        f.write(output)

In [ ]:
warnings.filterwarnings("ignore")
suite = openml.study.get_suite(99)
print(suite)
tasks = suite.tasks

In [ ]:
def train_ebm_model(dataset, n_jobs=-1, random_state=RANDOM_STATE, interactions=0, debug=False):
    model_clf = ExplainableBoostingClassifier(random_state=RANDOM_STATE, n_jobs=-1, feature_types=ds.feature_types, interactions=0)
    model_clf.fit(ds.df_train[ds.features], ds.df_train[ds.target])
    
    y_pred = model_clf.predict(ds.df_test[ds.features])
    y_test = ds.df_test[ds.target]
    if debug:
        print(f"Model accuracy: {round(accuracy_score(y_test, y_pred),2)}")
    
    return model_clf

In [ ]:
def train_causal_model(dataset):
    causal_model = lingam.DirectLiNGAM()
    causal_model.fit(ds.df_train[ds.features])
    
    return causal_model

In [ ]:
class CCStats:
    def __init__(self):
        self.stats = []
    
    @staticmethod
    def get_columns():
        return ['model_name','dataset','fidelity','probability','loss','proximity_loss','sparsity_loss','causality_loss','diversity',
                'no_ov_cfs','ov_loss','ov_proximity_loss','ov_sparsity_loss','ov_causality_loss','ov_diversity'
                ,'execution_time']
    
    def get_stats_for_dataset(self, name):
        datasum = pd.DataFrame(self.stats, columns=self.get_columns())
        
        return datasum[datasum['dataset']==name].groupby('model_name').mean().to_string(header=True)
    
    def get_total_stats(self):
        return pd.DataFrame(self.stats, columns=self.get_columns())
    
    def save_total_stats(self, filename):
        self.get_total_stats().to_csv(filename, index=False)

    def append_stats(self, method, cfs, dataset, explain_instance, model_clf, casual_model, desired_class, pbounds, execution_time):
        """
        Append statistical evaluation metrics to the global `stats` list.

        This function evaluates various metrics for a given set of counterfactuals (cfs) and appends the results to a global list named `stats`. The evaluation metrics include accuracy, y-loss, proximity loss, sparsity loss, causal penalty, and diversity loss.

        Parameters:
        name (str): The name or identifier for the set of counterfactuals being evaluated.
        cfs (np.ndarray): An array of counterfactual instances to be evaluated.

        Metrics Evaluated:
        - Accuracy: The mean accuracy of the model's predictions on the counterfactuals.
        - y-Loss: The mean y-loss, which measures the difference between the model's predicted and desired classes.
        - Proximity Loss: The mean proximity loss, which measures the distance between the counterfactuals and the original instance being explained.
        - Sparsity Loss: The mean sparsity loss, which measures how many features differ between the counterfactuals and the original instance.
        - Causal Penalty: The mean causal penalty, which assesses the impact of the counterfactuals on a causal model.
        - Diversity: A measure of diversity among the counterfactuals.

        The results are appended as a list to the global `stats` list in the following order:
        [name, dataset,accuracy, mean_y_loss, mean_proximity_loss, mean_sparsity_loss, mean_causal_penalty, diversity,execution_time]

        Returns:
        List

        Example:
        >>> name = "Counterfactual Set 1"
        >>> cfs = np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
        >>> append_stats(name, cfs)
        """
        categorical_indicator = dataset._categorical_indicator

        record=[]
        record.append(method)
        record.append(dataset.name)
        if cfs is not None:
            record.append(np.mean(desired_class==model_clf.predict(cfs)))
            record.append(np.mean([p[desired_class] for p in model_clf.predict_proba(cfs)]))
            record.append(np.mean([compute_yloss(model_clf,ce, desired_class)[0] for ce in cfs]))
            record.append(np.mean([compute_proximity_loss(ce.reshape(1,-1), explain_instance,pbounds=pbounds,features_order=dataset.features,categorical=categorical_indicator) for ce in cfs]))
            record.append(np.mean([compute_sparsity_loss(ce.reshape(1,-1),explain_instance) for ce in cfs]))
            record.append(np.mean([compute_causal_penalty(ce.reshape(1,-1),casual_model.adjacency_matrix_,casual_model.causal_order_,categorical=categorical_indicator) for ce in cfs]))
            record.append(compute_diversity_loss(cfs))
            
            mask = (desired_class==model_clf.predict(cfs))
            ovcfs = [cfs[i] for i in range(len(cfs)) if mask[i] ]
            
            if len(ovcfs) > 0:
                record.append(len(ovcfs))
                record.append(np.mean([compute_yloss(model_clf,ce, desired_class)[0] for ce in ovcfs]))
                record.append(np.mean([compute_proximity_loss(ce.reshape(1,-1), explain_instance,pbounds=pbounds,features_order=dataset.features,categorical=categorical_indicator) for ce in ovcfs]))
                record.append(np.mean([compute_sparsity_loss(ce.reshape(1,-1),explain_instance) for ce in ovcfs]))
                record.append(np.mean([compute_causal_penalty(ce.reshape(1,-1),casual_model.adjacency_matrix_,casual_model.causal_order_,categorical=categorical_indicator) for ce in ovcfs]))
                record.append(compute_diversity_loss(ovcfs))
            else:
                record.append(0)
                record.append(np.nan)
                record.append(np.nan)
                record.append(np.nan)
                record.append(np.nan)
                record.append(np.nan)
            
            record.append(execution_time)
        else:
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
            record.append(np.nan)
        
        self.stats.append(record)

        return record

In [ ]:
from sklearn.model_selection import train_test_split
import traceback
import sys
sys.path.append('./recourse/model/LORE')
sys.path.append('./recourse/model')
from LORE import lore
from LORE.neighbor_generator import *
from recourse.utils import utils
from cfnow import find_tabular
from alibi.explainers import CounterfactualProto
from alibi.explainers import CEM
from alibi.explainers import Counterfactual


class ExplainersRegistry:
    def __init__(self, model_clf, causal_model, dataset, num_cfs, stats, time_limit=1800, n_iter=100, init_points=500):
        self.model_clf = model_clf
        self.causal_model = causal_model
        self.ds = dataset
        self.num_cfs = num_cfs
        self.stats = stats
        self.time_limit = time_limit
        self.n_iter = n_iter
        self.init_points = init_points        
        
        self.pbounds = self.calc_pbounds()
        self.desired_class = None

        self.recipes = {}
        self.register_explainer('CCF', self._method_ccf)
        self.register_explainer('CCF-no-causal', self._method_ccf_no_causal)
        self.register_explainer('Dice', self._method_dice)
        self.register_explainer('Baseline', self._method_baseline)
        self.register_explainer('LUX', self._method_lux)
        self.register_explainer('LORE', self._method_lore)
        self.register_explainer('CEM', self._method_cem)
        self.register_explainer('CFProto', self._method_protocf)
        self.register_explainer('Wachter', self._method_wachter)
        self.register_explainer('cfnow', self._method_cfnow)
    
        
    def _method_baseline(self):
        ebcf = EBMCounterOptimizer(self.model_clf, pd.DataFrame(self.explain_instance.reshape(1,-1), columns=self.ds.features))
        cfs=[]
        for i in range(self.num_cfs):
            proba, cf = ebcf.optimize_proba(self.desired_class, feature_masked=self.ds.features)        
            cfs.append(cf)
        return pd.DataFrame(cfs).values
        
    def _method_ccf(self):
        return run_ccf(self.explain_instance, self.model_clf, self.ds, 
                       self.desired_class, self.num_cfs, self.causal_model, self.pbounds,
                      init_points=self.init_points, n_iter=self.n_iter)
        
    def _method_ccf_no_causal(self):
        return run_ccf(self.explain_instance, self.model_clf, self.ds, 
                       self.desired_class, self.num_cfs, self.causal_model, self.pbounds, as_causal=False,
                      init_points=self.init_points, n_iter=self.n_iter)
    
    def _method_cem(self):
        Xtr = self.ds.df_train[self.ds.features].values
        mode = 'PN'  # 'PN' (pertinent negative) or 'PP' (pertinent positive)
        shape = (1,) + Xtr.shape[1:]  # instance shape
        kappa = .2  # minimum difference needed between the prediction probability for the perturbed instance on the
                    # class predicted by the original instance and the max probability on the other classes
                    # in order for the first loss term to be minimized
        beta = .1  # weight of the L1 loss term
        c_init = 10.  # initial weight c of the loss term encouraging to predict a different class (PN) or
                      # the same class (PP) for the perturbed instance compared to the original instance to be explained
        c_steps = 10  # nb of updates for c
        max_iterations = 1000  # nb of iterations per value of c
        feature_range = (Xtr.min(axis=0).reshape(shape)-.1,  # feature range for the perturbed instance
                         Xtr.max(axis=0).reshape(shape)+.1)  # can be either a float or array of shape (1xfeatures)
        clip = (-1000.,1000.)  # gradient clipping
        lr_init = 1e-2  # initial learning rate

        cem = CEM(self.model_clf.predict_proba, 
          mode, 
          shape, 
          kappa=kappa, 
          beta=beta, 
          feature_range=feature_range,
          max_iterations=max_iterations, 
          c_init=c_init, 
          c_steps=c_steps,
          learning_rate_init=lr_init, 
          clip=clip)

        cem.fit(Xtr, no_info_type='median')  # we need to define what feature values contain the least
                                                 # info wrt predictions
                                                 # here we will naively assume that the feature-wise median
                                                 # contains no info; domain knowledge helps!
        explanation = cem.explain(self.explain_instance.reshape(1,-1), verbose=False)
        if explanation['PN'] is not None:
            cfs = [explanation['PN'].ravel() for _ in range(self.num_cfs)]
        else:
            raise Exception('No CF found')
        return cfs

    def _method_protocf(self):
        Xtr = self.ds.df_train[self.ds.features].values
        shape = (1,) + Xtr.shape[1:]
        beta = .01
        c_init = 1.
        c_steps = 5
        max_iterations = 500
        rng = (-1., 1.)  # scale features between -1 and 1
        rng_shape = (1,) + Xtr.shape[1:]
        feature_range = (Xtr.min(axis=0).reshape(shape)-.1,  # feature range for the perturbed instance
                         Xtr.max(axis=0).reshape(shape)+.1)  # can be either a float or array of shape (1xfeatures)
        cat_vars_ord={}
        for i,ci in enumerate(self.ds.categorical_indicator):
            if ci:
                cat_vars_ord[i]=len(np.unique(Xtr[:,i]))
        cf = CounterfactualProto(self.model_clf.predict_proba,
                 shape,
                 beta=beta,
                 cat_vars=cat_vars_ord,
                 max_iterations=max_iterations,
                 feature_range=feature_range,
                 c_init=c_init,
                 c_steps=c_steps,
                 eps=(.01, .01)  # perturbation size for numerical gradients
                )
        cf.fit(Xtr, d_type='abdm', disc_perc=[25, 50, 75])
        explanation = cf.explain(self.explain_instance.reshape(1,-1),
                                 target_class=[self.desired_class])
        if explanation['cf'] is not None:
            cfs = [explanation['cf']['X'].ravel() ]
        else:
            raise Exception('No CF found')
        return cfs
        
    def _method_wachter(self):
        Xtr = self.ds.df_train[self.ds.features].values
        shape = (1,) + Xtr.shape[1:]
        beta = .01
        c_init = 1.
        c_steps = 5
        max_iterations = 500
        rng = (-1., 1.)  # scale features between -1 and 1
        rng_shape = (1,) + Xtr.shape[1:]
        feature_range = ((np.ones(rng_shape) * rng[0]).astype(np.float32),
                         (np.ones(rng_shape) * rng[1]).astype(np.float32))




        cf = Counterfactual(self.model_clf.predict_proba, shape, distance_fn='l1', target_proba=1.0,
                            target_class=int(self.desired_class), max_iter=1000, early_stop=50, lam_init=1e-1,
                            max_lam_steps=10, tol=0.05, learning_rate_init=0.1,
                            feature_range=feature_range, eps=0.01, init='identity',
                            decay=True, write_dir=None, debug=False)
        
        explanation = cf.explain(self.explain_instance.reshape(1,-1))
        if explanation['cf'] is not None:
            cfs= [explanation['cf']['X'].ravel()]
        else:
            raise Exception('No CF found')
        return cfs
    
    def _method_lux(self):
        Xtr,_ = train_test_split(self.ds.df_train.copy(), train_size=min(self.ds.df_train.shape[0],2000)) 
        cols = [f.replace('-','') for f in Xtr.columns]
        Xtr.columns =  cols

        features = [f for f in cols if f not in ds.target]
        explain_instance = self.explain_instance.reshape(1,-1)
        lux = LUX(predict_proba=self.model_clf.predict_proba, 
                  #classifier=model_clf, 
                  neighborhood_size=0.1)
        lux.fit(Xtr[features],Xtr[self.ds.target],instance_to_explain=explain_instance, categorical=self.ds._categorical_indicator, n_jobs=-1)
        
        _,Xs = train_test_split(Xtr[features],stratify=Xtr[self.ds.target], test_size=1000)
        
        cf_lux = lux.counterfactual(np.array(explain_instance), Xs[features], counterfactual_representative='nearest', topn=self.num_cfs)
        cfs = [np.array(c['counterfactual']) for c in cf_lux]
        
        return cfs
    
    def _method_dice(self):
        hyperparams = {"num": self.num_cfs, "desired_class": int(self.desired_class), "posthoc_sparsity_param": 0.1}
        dice = Dice(self.model_clf, data=self.ds, hyperparams=hyperparams)

        return dice.get_counterfactuals(pd.DataFrame([self.explain_instance], columns=self.ds.features)).values
    
    def _method_cfnow(self):
        try:
            local_time_limit = int(self.cfgen_time_agg/self.cfgen_methods_no)
            if local_time_limit < 20:
                # make it reasonable limit for this kind of search algorithm
                local_time_limit = 20
        except:
            local_time_limit=20
            
        cf_obj = find_tabular(
            factual=pd.Series(self.explain_instance),
            count_cf=self.num_cfs,
            feat_types={i: 'cat' if self.ds.categorical_indicator[i] else 'cont' for i in range(len(self.ds.categorical_indicator))},
            model_predict_proba=self.model_clf.predict_proba,
            limit_seconds= local_time_limit)
        cfs= list(cf_obj.cfs[:self.num_cfs]) if cf_obj.total_cf >= self.num_cfs else list(cf_obj.cfs)
        return cfs
        
    
    def _method_lore(self):
        Xtr= self.ds.df_train.copy()
        cols = [f.replace('-','') for f in Xtr.columns]
        Xtr.columns =  cols
        features = [f for f in cols if f not in self.ds.target]
        
        myds = utils.prepare_ds_lore(Xtr,discrete=self.ds.categorical_indicator,class_name=self.ds.target_class)
        X_explain = np.concatenate(( self.explain_instance.reshape(1,-1), myds['X']))
        try:
            exp_LORE, info_LORE = lore.explain(0, X_explain,
                                               myds, self.model_clf,
                                               ng_function=genetic_neighborhood,
                                               discrete_use_probabilities=True,
                                               continuous_function_estimation=False,
                                               returns_infos=True, path='./recourse/model/LORE/yadt/',
                                               sep=';', log=True, depth=100)
        except:
            #Try numerical only
            myds = utils.prepare_ds_lore(Xtr,class_name=self.ds.target_class)
            X_explain = np.concatenate(( self.explain_instance.reshape(1,-1), myds['X']))
            exp_LORE, info_LORE = lore.explain(0, X_explain,
                                               myds, self.model_clf,
                                               ng_function=genetic_neighborhood,
                                               discrete_use_probabilities=True,
                                               continuous_function_estimation=False,
                                               returns_infos=True, path='./recourse/model/LORE/yadt/',
                                               sep=';', log=True, depth=100)
    
        query = ' & '.join([f'({str(a)} {str(b)})' if str(a) not in str(b) else f'({b})' for a,b in exp_LORE[1][0].items()])
        cfs = Xtr.query(query)
        fcfs = cfs.sample(min(len(cfs), self.num_cfs))
        return list(fcfs[features].values)
    
    def register_explainer(self, name, func):
        self.recipes.update({name: func})
        
    def calc_pbounds(self):
        return {f: (self.ds.df[f].min(), self.ds.df[f].max()) for f in self.ds.features}
        
    def set_desired_class(self, explain_instance, desired_class=None):
        if desired_class:
            self.desired_class = desired_class
        else:
            self.desired_class = (self.model_clf.predict(explain_instance)[0]+1) % self.ds.df_train[self.ds.target].nunique()
        
    def run_explainers(self, explain_instance):
        self.set_desired_class(explain_instance)
        self.explain_instance = explain_instance
        self.cfgen_methods_no=0
        self.cfgen_time_agg=0
        tf.keras.backend.clear_session()
        for method_name, method_func in self.recipes.items():
            print(f'Running {method_name}...')
            try:
                ts = tstime()
                signal.alarm(self.time_limit)
                cfs = method_func()
                te = tstime()-ts
                self.cfgen_methods_no+=1
                self.cfgen_time_agg+=te
                self.stats.append_stats(method_name, cfs, self.ds, self.explain_instance, self.model_clf, self.causal_model, int(self.desired_class), self.pbounds, te)
                print('Done (in {:.2f}s)'.format(te))                
            except TimeoutError:
                print('Timeout...')
                self.stats.append_stats(method_name, None, self.ds, self.explain_instance, self.model_clf, self.causal_model, int(self.desired_class), self.pbounds, np.nan)
            except Exception as e:
                print(f"An error occurred: {e}")
                traceback.print_exc()  # This prints the full traceback
                
                self.stats.append_stats(method_name, None, self.ds, self.explain_instance, self.model_clf, self.causal_model, int(self.desired_class), self.pbounds, np.nan)
            signal.alarm(0)
                


In [ ]:
import os
import pickle

def load_or_dump_cached_file(cache_dir, cached_file_name, cached_data=None):
    """
    Load cached data from the specified file, or if cached_data is provided, 
    dump it into the file in the specified directory. 
    
    If the file does not exist or loading fails, and cached_data is not provided, return None.

    Parameters:
    cache_dir (str): Directory where the cache file is stored.
    cached_file_name (str): Name of the cache file.
    cached_data (object, optional): Data to be cached (default is None).

    Returns:
    object or None: The loaded cached data, or None if loading is not possible and cached_data is None.
    """
    # Ensure the directory exists
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    
    # Construct the full file path
    file_path = os.path.join(cache_dir, cached_file_name)

    # If cached_data is provided, dump it into the file
    if cached_data is not None:
        with open(file_path, 'wb') as file:
            pickle.dump(cached_data, file)
        return cached_data

    # Otherwise, try loading the cached data
    if os.path.exists(file_path):
        try:
            with open(file_path, 'rb') as file:
                return pickle.load(file)
        except (pickle.UnpicklingError, EOFError, FileNotFoundError):
            return None
    else:
        return None


## Evaluation starts here

In [ ]:
stats = CCStats()
log2file('', clear=True)
SAMPLE_SIZE = 10
NUM_CFS = 3
init_points = 200
n_iter = int(0.2*init_points)

# Set the time limit in seconds for a calculation of one cfs
time_limit = 60*30  # seconds (10 minutes)
model_time_limit = 3600 # hour

use_suite = False

if not use_suite:
    all_datasets = openml.datasets.list_datasets(output_format='dataframe')
    classification_datasets = all_datasets[(all_datasets['NumberOfClasses']>1) & (all_datasets['NumberOfInstances']>2000) & 
                                           (all_datasets['NumberOfMissingValues']==0) & 
                                           #(all_datasets['NumberOfInstances']< 10000)& 
                                           (all_datasets['NumberOfFeatures'] < 30)].drop_duplicates(subset=['name'])
    classification_datasets['name'] = classification_datasets['name'].apply(lambda x: x.split('_')[0])
    classification_datasets=classification_datasets.drop_duplicates(subset=['name'])
    
    tasks = classification_datasets['did']

In [ ]:
# OpenML datasets
import warnings
warnings.filterwarnings('ignore')
cache_dir = './cache/'
for task_id in tasks:
    try:
        ds = load_or_dump_cached_file(cache_dir, f'dataset-{task_id}.pkl', cached_data=None)
        if ds is None:
            ds = OpenmlData(openml.datasets.get_dataset(task_id))
            load_or_dump_cached_file(cache_dir, f'dataset-{task_id}.pkl', cached_data=ds)
            
        log2file(f'Processing ID: {task_id} for name: {ds.name}\n')
        print(f'{"*" * 50} {ds.name} {"*" * 50}')

        print('Training classification model...', end='')
        model_clf = load_or_dump_cached_file(cache_dir, f'model_clf-{task_id}.pkl', cached_data=None)
        if model_clf is None:
            model_clf = train_ebm_model(ds, debug=True)
            load_or_dump_cached_file(cache_dir, f'model_clf-{task_id}.pkl', cached_data=model_clf)  
        print('Done')

        try:
            signal.alarm(model_time_limit) # one hour for causal model
            print('Training casual model...', end='')
            
            causal_model = load_or_dump_cached_file(cache_dir, f'causal_model-{task_id}.pkl', cached_data=None)
            if causal_model is None:
                causal_model = train_causal_model(ds)
                load_or_dump_cached_file(cache_dir, f'causal_model-{task_id}.pkl', cached_data=causal_model)  
            print('Done.')
            signal.alarm(0)
        except TimeoutError:
            print('Timeout, aborting, moving to another dataset...')
            continue

        explainers_registry = ExplainersRegistry(model_clf, causal_model, ds, NUM_CFS, stats, time_limit,
                                                init_points = init_points, n_iter = n_iter)
        print('Calculating counterfactuals...')
        for explain_instance in ds.sample_test(SAMPLE_SIZE):
            explainers_registry.run_explainers(explain_instance)
        log2file(stats.get_stats_for_dataset(ds.name))
    except Exception as e:
        # Capture the traceback as a string
        stack_trace = traceback.format_exc()
        log2file(stack_trace)
        print(stack_trace)
